In [1]:
!pip install torch datasets transformers sacrebleu accelerate -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, MarianMTModel, AdamW, get_scheduler
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Helsinki-NLP/opus_books", "en-hu")

# Split the dataset
train_dataset = dataset['train'].select(range(5000))
val_dataset = dataset['train'].select(range(5000, 6000))
test_dataset = dataset['train'].select(range(6000, 7000))

# View the number of elements in each dataset
print("Number of elements in the train dataset:", train_dataset.num_rows)
print("Number of elements in the validation dataset:", val_dataset.num_rows)
print("Number of elements in the test dataset:", test_dataset.num_rows)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hu")

# Preprocess function
def preprocess_function(examples):
    inputs = [ex['en'] for ex in examples['translation']]
    targets = [ex['hu'] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=["translation"])
tokenized_val = val_dataset.map(preprocess_function, batched=True, remove_columns=["translation"])
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=["translation"])

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/137151 [00:00<?, ? examples/s]

Number of elements in the train dataset: 5000
Number of elements in the validation dataset: 1000
Number of elements in the test dataset: 1000


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
class TranslationDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.input_ids = tokenized_dataset["input_ids"]
        self.attention_mask = tokenized_dataset["attention_mask"]
        self.labels = tokenized_dataset["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        item = {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "attention_mask": torch.tensor(self.attention_mask[idx]),
            "labels": torch.tensor(self.labels[idx])
        }
        return item

train_dataset = TranslationDataset(tokenized_train)
val_dataset = TranslationDataset(tokenized_val)
test_dataset = TranslationDataset(tokenized_test)

# View the number of elements in the tokenized datasets
print("Number of elements in the tokenized train dataset:", len(train_dataset))
print("Number of elements in the tokenized validation dataset:", len(val_dataset))
print("Number of elements in the tokenized test dataset:", len(test_dataset))


Number of elements in the tokenized train dataset: 5000
Number of elements in the tokenized validation dataset: 1000
Number of elements in the tokenized test dataset: 1000


In [4]:
# Load the model
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hu")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [5]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,No log,0.532157
2,0.613500,0.527720
3,0.613500,0.528064


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62521]], 'forced_eos_token_id': 0}


TrainOutput(global_step=939, training_loss=0.544838447286425, metrics={'train_runtime': 388.1498, 'train_samples_per_second': 38.645, 'train_steps_per_second': 2.419, 'total_flos': 508475473920000.0, 'train_loss': 0.544838447286425, 'epoch': 3.0})

In [6]:
metrics = trainer.evaluate(test_dataset)
print(metrics)

{'eval_loss': 0.6199615597724915, 'eval_runtime': 6.9899, 'eval_samples_per_second': 143.064, 'eval_steps_per_second': 9.013, 'epoch': 3.0}


In [10]:
# Kiểm tra nếu GPU có sẵn và chuyển model sang GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Câu đầu vào
input_text = "Source: Project GutenbergAudiobook available here"

# Tokenize câu đầu vào và chuyển sang cùng thiết bị với model
inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
inputs = {key: value.to(device) for key, value in inputs.items()}

# Tạo bản dịch
translated_tokens = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

# Giải mã các token đã dịch
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

print("Translated text:", translated_text)

Translated text: Forrás: Project GutenbergAudiobook elérhető itt
